In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("JoinRDD_CoGroup").getOrCreate()
spark.sparkContext.setLogLevel("WARN")

21/12/02 09:34:27 WARN Utils: Your hostname, duyvan-Inspiron-5402 resolves to a loopback address: 127.0.1.1; using 10.0.130.75 instead (on interface wlp44s0)
21/12/02 09:34:27 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
21/12/02 09:34:28 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
21/12/02 09:34:28 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [2]:
sc = spark.sparkContext

In [4]:
ord = sc.textFile('orders')
ordItems = sc.textFile('order_items')

In [5]:
ord.take(5)

['1,2013-07-25 00:00:00.0,11599,CLOSED',
 '2,2013-07-25 00:00:00.0,256,PENDING_PAYMENT',
 '3,2013-07-25 00:00:00.0,12111,COMPLETE',
 '4,2013-07-25 00:00:00.0,8827,CLOSED',
 '5,2013-07-25 00:00:00.0,11318,COMPLETE']

In [5]:
ordMap=ord.map( lambda x: (x.split(',')[0], x.split(',')[2]) )

In [7]:
ordItemMap=ordItems.map( lambda x: (x.split(',')[1], x.split(',')[4]) )

In [6]:
ordMap.take(5)

[('1', '11599'), ('2', '256'), ('3', '12111'), ('4', '8827'), ('5', '11318')]

In [8]:
ordItemMap.take(5)

[('1', '299.98'),
 ('2', '199.99'),
 ('2', '250.0'),
 ('2', '129.99'),
 ('4', '49.98')]

In [9]:
findSubtotalForCust = ordMap.join(ordItemMap)

In [10]:
findSubtotalForCust.take(5)

[('4', ('8827', '49.98')),
 ('4', ('8827', '299.95')),
 ('4', ('8827', '150.0')),
 ('4', ('8827', '199.92')),
 ('10', ('5648', '199.99'))]

In [13]:
findSubtotalForCust.map(lambda x : x[1][0] + ',' + x[1][1] ).first()

'8827,49.98'

In [3]:
tuple1 = ("a", 1)
tuple2 = ("b", 4)
tuple3 = ("a", 2)

In [16]:
x = sc.parallelize([tuple1, tuple2])

In [17]:
y = sc.parallelize([tuple3])

In [11]:
x.take(5)

[('a', 1), ('b', 4)]

In [22]:
# x = sc.parallelize([("a", 1), ("b", 4)])

In [21]:
# y = sc.parallelize([("a", 2)])

In [20]:
# [(x, tuple(map(list, y))) for x, y in sorted(list(x.cogroup(y).collect()))]

In [18]:
xy = x.cogroup(y)

In [19]:
xy.take(2)

[('b',
  (<pyspark.resultiterable.ResultIterable at 0x7f4561270d30>,
   <pyspark.resultiterable.ResultIterable at 0x7f45612709a0>)),
 ('a',
  (<pyspark.resultiterable.ResultIterable at 0x7f4560fdf790>,
   <pyspark.resultiterable.ResultIterable at 0x7f4560fdf760>))]

In [27]:
for i, j in xy.collect():
    print(i, tuple(map(list, j)) )

b ([4], [])
a ([1], [2])


In [28]:
rdd = sc.parallelize([1,2,3])

In [30]:
sorted(rdd.cartesian(rdd).collect())

[(1, 1), (1, 2), (1, 3), (2, 1), (2, 2), (2, 3), (3, 1), (3, 2), (3, 3)]

In [ ]:
# for i in xy.take(2):
#     print(i)

In [31]:
spark.stop()
